In [1]:
import numpy as np 
import scipy as sp 
import matplotlib as plt
import torch

# Sum and Product rules of probability

The sum and product rules of probability are the backbone of bayesian machine learning. They are what generally facilitate the computation or at least exposition of how to compute a posterior distribution.
<br>Sum Rule: $P(A) = \sum_{b \in B} P(A, B)$ for discrete case and $P(A) = \int_{b \in B} P(A,B) db$ for the continuous case
<br>Product Rule: $P(A,B) = P(A | B)P(B) = P(B|A)P(A)$
<br> Both of these results are a conected to Bayes rule which states,
$$P(A|B) = \frac{P(A, B)}{P(B)} =\frac{P(B|A)P(A)}{P(B)} = \frac{P(B|A)P(A)}{\sum_{a \in A}P(B|A)}$$
A couple of interpretations of this follow.
1. (The First Equality) The conditional probability of A given B is equal to all of the events where A and B occur together divide by the a reduced sample space of events where only B occurs. 
2. (The Second Equality) The posterior probability of A given an observation of B, $P(A|B)$, is equal to the prior probability of A, P(A), weighted by the likelihood of A given B $P(B|A)= L(A|B)$. This is the perspective that we tend to take in Bayesian Machine Learning.  

# Where do they come up? The Three Approaches

Answer: Everywhere!
The sum and product rules come up in some form or another any time we are trying to fit a model to data. The example that follows is the case of linear regression. Where our design matrix $X \in \mathbb{R}^{N x D}$ so we have N samples of D dimensional data. We then specify a prior over our parameter vector $\mathbf{w} = \mathcal{N}(0,\alpha^2 \mathbf{I})$ where $\mathbf{w}\in\mathbb{R}^D$. Our noise for this model is assumed to be Gaussian, $\epsilon_i = \mathcal{N}(0, \sigma^2)$ Our observation model then takes the form of,
$$y_i = w_o + w_1x_1 + w_2x_2 + \dots + w_dx_d + \epsilon_i$$
In matrix notation this is, 
$$\mathbf{Y} =\mathbf{X}\mathbf{w} + \epsilon$$

We can see the way the sum and product rules come up in the three primary approaches to trying to fit our model to the data. The first two approaches will construct a function $\mathbf{Y}(\mathbf{X, w})$ by selecting one setting of the parameter $\mathbf{w}$. The last, fully bayesian treatment of the problem will instead construct a posterior predictive distribution $P(y^*|x^*,\mathbf{X},\mathbf{Y}, \sigma^2, \alpha^2)$ where $x^*$ is a new data point we have observed and want to make a prediction $y^*$ on. 

## 1. Maximum Likelihood 
This first approach seeks to do one thing and one thing only. Find the setting of parameters that maximized the probability of having observed the data given our observation model. This is a task that does not depend on the prior over w so we will ignore that for now. To do this we first note that $\mathbf{Y} \sim \mathcal{N}(\mathbf{w}^T \mathbf{X}, \sigma^2 \mathbf{I})$ If we assume sigma is known, if not we know the unadjusted sample variance is the MLE for that.  
$$L(w|\mathcal{D}, \sigma^2) = \frac{1}{(2\pi)^{N/2}|\sigma^2 I|^{\frac{1}{2}}}\exp(-\frac{1}{2}(\mathbf{Y}-\mathbf{X}\mathbf{w})^T\frac{1}{\sigma^2}\mathbf{I}(\mathbf{Y}-\mathbf{X}\mathbf{w}))$$
$$\implies \log(L(w|\mathcal{D}, \sigma^2)) = -\frac{N}{2}\log(2\pi) - \frac{1}{2}\log(|\sigma\mathbf{I}|) -\frac{1}{2}(\mathbf{Y}-\mathbf{X}\mathbf{w})^T\frac{1}{\sigma^2}\mathbf{I}(\mathbf{Y}-\mathbf{X}\mathbf{w})$$
Differentiating wrt w and setting to 0 yields,
$$0 = -\frac{1}{\sigma^2}\mathbf{X}^T(\mathbf{Y}-\mathbf{X}\mathbf{w}) = -\frac{1}{\sigma^2}(\mathbf{X}^T\mathbf{Y}-\mathbf{X}^T\mathbf{X}\mathbf{w})$$
$$\implies \mathbf{X}^T\mathbf{X}\mathbf{w} = \mathbf{X}^T\mathbf{Y} \implies \hat{\mathbf{w}}_{MLE} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{Y}^T$$

From here any new predictions on $\mathbf{x}^*$ would simply be pluged into the function $y(\mathbf{x},\mathbf{w}) = \mathbf{x^*}\hat{\mathbf{w}}_{MLE}$. The important thing to note here is that this learning approach only depends on the data that we have seen, through the likelihood function. The maximization of the likelihood will then put bet all future decisions for the model on one setting of parameters. This can prove to be unrealistic at times particularly in the presence of small datasets(As the MLE under regularity conditions is a consistent estimator ie. it approaches the true parameter value in the lim as n -> inf). An example is the task of flipping a coin. This is the setting of a Bernoulli Random Variable. If we only conduct one flip and the outcome is tails the MLE for the parameter p will be equal to 1 ie. the MLE would say that we are certain the next flip is also going to be tails. This is of course unrealistic and the next approach seeks to remedy this by incorporating our prior beliefs about the model into a partially bayesian framework

## 2. Maximum A Posteriori 
In this approach we see to maximize the posterior distribution over the parameters. A posterior distribution by definition depends on a prior distribution. In this case the use of the product rules of probability is crucial. A direct application of this rule yields
$$P(w|\mathcal{D}, \alpha^2) \propto P(\mathcal{D} | \mathbf{w}, \sigma^2, \alpha^2)P(\mathbf{w}|\alpha^2)$$
The right hand side here is actually equal to the joint distribtion of D and w but for the case of maximization the normalizing denominator is unecessary because it depends only on D. 
$$\log(P(w|\mathcal{D}, \alpha^2)) \propto (\mathbf{Y}-\mathbf{X}\mathbf{w})^T\frac{1}{\sigma^2}\mathbf{I}(\mathbf{Y}-\mathbf{X}\mathbf{w}) + \frac{1}{\alpha^2}\mathbf{w}^T\mathbf{w}$$
We can maximize this directly or note that the posterior is also Gaussian and specify the mean and covariance matrix,
$$\implies log(P(w|\mathcal{D}, \alpha^2)) \propto \frac{1}{\sigma^2}(\mathbf{Y}^T\mathbf{Y}-2\mathbf{X}^T\mathbf{w}^T\mathbf{Y} + \mathbf{w}^T\mathbf{X}^T\mathbf{X}\mathbf{w}) + \frac{1}{\alpha^2}\mathbf{w}^T\mathbf{w}$$
Collecting the quadratic terms, linear terms and droping what doesn't depend on w,
$$ Cov(\mathbf{w}|\mathcal{D}, \alpha^2)^{-1} = \frac{1}{\sigma^2}\mathbf{X}^T\mathbf{X} + \frac{1}{\alpha^2}\mathbf{I} = S^{-1}$$
Solving, 
$$S^{-1}\mu = \frac{1}{\sigma^2}\mathbf{X}^T\mathbf{w}^T\mathbf{Y} \implies \mu = S\frac{1}{\sigma^2}\mathbf{X}^T\mathbf{w}^T\mathbf{Y}$$

Thus fully specifiying our posterior distribution. The mean specifiying the maximum of our posterior is given by, 
$$\mathbf{w_{MAP}} = ((\alpha^2 \mathbf{I})^{-1} + \mathbf{X}(\sigma^2 \mathbf{I})^{-1}\mathbf{X^T})^{-1}(\mathbf{X}(\sigma^2 \mathbf{I})^{-1}\mathbf{y})$$
This approach in this setting results in a regularized least squares solution to the regression problem. The regularization here is the result of our prior belief that the parameters ome from a normal distribution.

## 3. Posterior Predictive 

This qualifies as the fully Bayesian treatment of the problem where after using the product rule to derive the posterior of the parameters we use it again in conjunction with the sum rule to arrive at a predictive posterior distribution. $$P(y^*|\mathcal{D}, x^*, \sigma^2, \alpha^2) = \int_{w \in W} P(y^*|x^*, \mathbf{w}, \sigma^2, \alpha^2)P(\mathbf{w}|\mathcal{D}, \sigma^2, \alpha^2)d\mathbf{w}$$ The derivation of this proceeds similarly to above. The benefit of this is a full treatment of the uncertainty of our model. The solution incorporate the uncertainty from the posterior of the parameters alongside the uncertainty from the model itself. 


# The Way of Bayes (BMA, Evidence Function, Occam's Razor)

Occams razor in the context of Bayesian Modeling is the idea that even though many models may be able to fit the Data well, there is a model that most parsimoniously explains the data. Mathematicall this is expressed in through the conditional probability $P(\mathcal{M}|\mathcal{D})$ where M is model and D is the data. This corresponds to the likelihood for that the Data came from model M. The Bias-Variance tradeoff is implicit in this idea, as we are looking for a model that explains the data well, while not beign overly flexible. Using the product rule for a particular model i,
$$P(\mathcal{M}_i|\mathcal{D}) \propto P(\mathcal{D}|\mathcal{M}_i)P(\mathcal{M_i})$$
The interesting quantity here is known as the evidence function $P(\mathcal{D}|\mathcal{M}_i)$ this expresses the preference shown by the data for different models. It is analogous to the likelihood of a model given the Data. These ideas together give us Bayesian Model Averaging. The idea that we can use multiple models together to come up with our predictive distribution by weighting each model by how well it explains the data. In fact we have already done this in the fully Bayesian treatment of linear regression. By marginalizing over w we in effect take all possible distributions weighted by their probability given the data and sum them together. More generally, 
$$P(y|x, \mathcal{D}) = \sum_{i=1}^{N}P(y|x, \mathcal{M_i})P(\mathcal{M}_i|\mathcal{D})$$
While in some cases we can come upu with this mixture distribution directly(as in the posterior predictive for linear regression with gaussian noise and priors), often it is not feasible to compute this. Nontheless we still want ot use the idea of the evidence function to come up with a best model for our data, that is we want to use the evidence function to perform model selection. The evidence decomposes into, 

$$P(\mathcal{D}|\mathcal{M}_i) = \int P(\mathcal{D}|w, \mathcal{M}_i)P(w|\mathcal{M}_i)dw$$
$$P(w |\mathcal{D}, \mathcal{M_i}) \propto P(\mathcal{D}|w, \mathcal{M}_i)P(w|\mathcal{M}_i) $$
So the integrand is proportional to the posterior distribution of w. Using this and some convenient assumptions we can begin to parse how the evidence function controls model complexity and embodies Occam's razor. First we assume the posterior distribution, (The Integrand) is sharply peaked around its maximum. Using this we estimate the integral by the value of the integrand at this peak times the width of the posterior distribution $\sigma_{w|D}$. 
$$\implies P(\mathcal{D}|\mathcal{M}_i) \approx P(\mathcal{D}|w_{MAP}, \mathcal{M}_i)P(w_{MAP}|\mathcal{M}_i)\sigma_{w|D}$$
<br>evidence function =  best fit likelihood x occams factor
<br>Further we assume we for model $\mathcal{M_i}$ we have a uniform prior over the parameters $P(w|\mathcal{M}_i) = \frac{1}{\sigma_w}$ $$\implies P(\mathcal{D}|\mathcal{M}_i) \approx P(\mathcal{D}|w_{MAP}, \mathcal{M}_i)\frac{\sigma_{w|D}}{\sigma_w} $$ 
<br> Looking into this further we can see how he evidence function operates, $$\log P(\mathcal{D}|\mathcal{M}_i) \approx \log P(\mathcal{D}|w_{MAP}, \mathcal{M}_i) + \log\frac{\sigma_{w|D}}{\sigma_w}$$ $$\frac{\sigma_{w|D}}{\sigma_w} \lt 1 \implies \log\frac{\sigma_{w|D}}{\sigma_w} < 0$$ The posterior width is always going to be narrower than the prior because observation of data increases our confidence in a specific setting of parameters. Therefor we can see this occams factor as penalizing a model for having highly fine tuned parameters s.t. the posterior is almost certain. For a model governed by M parameters, assuming they all have the same ratio $\frac{\sigma_{w|D}}{\sigma_w}$ we get that $$\log P(\mathcal{D}|\mathcal{M}_i) \approx \log P(\mathcal{D}|w_{MAP}, \mathcal{M}_i) + M\log\frac{\sigma_{w|D}}{\sigma_w}$$ This makes it explicit how the complexity penalty increases with increasing numbers of parameters. Thus by maximizing the evidence we implicitly embody Occam's Razor, we are finding the model that fits the data well (has a high likelihood) while not being overly complex(Occams factor isn't too large).  

# Supervised Learning (Give me X and Y and I'll give you w)
The general idea of supervised learning is we have a data set of inputs and responses. We come up with a generative story for the data, then we maximize the likelihood, or the posterior, or find the predictive distribution from these. 

## K-Nearest Neighbors 
K Nearest Neighbors is a classification and regression algorithm. The algorithm conceptually is pretty simple, for a new training point x* we select the K nearest points irrespective of their class and whichever class is most represented is what we assign x* to. Probabilistically, we are interested in finding the posterior probability of $C_i$ given input x. $$P(C_i|x)= \frac{P(x|C_i)P(C_i)}{P(x)}$$ From our training data we can determine all of these. $$P(C_i) = n_i/n$$ The prior is just the number of training samples that belong to class i over total training data. $$P(x)=k/n$$ just the probability of k points being inside the sphere centered at x. Further the conditional probability $$P(x|C_i)= k_i/n_i$$ is just the number of points from class i in the sphere of K points. Putting this all together we get a posterior probability of $$\frac{k_i}{K}$$. The decision is then just whichever class has the highest posterior probability corresponding to the most members out of the K nearest neighbors.

## Neural Networks(Regression and Classsification)
### Part 1) The Network
The basic components of a neural network are linear combinations of input's, governed by a parameter vector or matrix, followed by nonlinear differentiable activation functions. By combining the linear combination with the activation function we arrive at a flexible sort of basis function where the parameters govern its form, we call this a layer. Composing these layers on top of each other results in what we call a multilayer neural network. In general we have a set of inputs $X \in \mathbb{R}^{NxD}$ and outputs $Y \in \mathbb{R}^{Nxp}$. The form of our response will dictate the form of our last layer. If the outputs are continuous values the last layer will be a linear layer, for classification the output would be a logistic funciton for 2 class problems and a softmax function for multiclass problems. At every layer we compute we have a set of M weights associated with our M hidden units in the layer. We then compute $$XW_{0} = Z, Z \in \mathbb{R}^{N\times D}$$ and apply our activation function $$h(Z)=Z'$$ Z' is then fed into the next layer where it is multiplied by that layers weights and fed into that layers activation function. The result of this iterative process is a function $$f(x,w), w = \{W_0, W_1, \dots, W_L\}$$ for L hidden layers where the function is just the composition of the hidden layers on top of each other. From this function depending on our task we can define a probabilistic model. 
<br> Regression: $y = f(x, w) + \epsilon$ where $\epsilon = \mathcal{N}(0, \sigma^2)$
<br>Classification: $P(y=1|f(x, w)) = \sigma(f(x,w))$ for binary classification and $P(y=j|f(x,w))=softmax_j(f(x,w))$
<br> From this we can come up with a likeilhood to maximize which will correspond to an error to minimize. Because of all of the non-linearities though this is a non-convex problem and hard to optimize. Which is where stochastic gradient decent and the computational graph come into play. 


### Part 2) The Optimization
So we have come up with our neural network architecture and now we want to figure out how to train it, but the problem is non-convex so all of our usual optimization algorithms fall short. That's where stochastic gradient decent and the structure of neural networks comes in. 

## K-Means Classification
For K-Means classification we have a set of N points in $\mathbb{R}^D$. Suppose they come from K-Classes we want ot model  

# Unsupervised Learning

## Discrete Latent Variables, EM Algorithm, Gaussian Mixture Models, and K-Means 

### K-Means
The basic idea behind K-Means is simple. Given a set of N data points in $\mathbb{R^D}$ we want to estimate K clusterings of the data. Intuitively for point $x_i$ to be in cluster k we want the distance between $x_i$ and the center of cluster k to be less than the distance from $x_i$ and any of the other cluster centers. We can write this problem as $$\underset{j}{\mathrm{argmin}}||\mathbf{x}_i - \mu_j||_2^2$$  where $\mu_i$ is the center of cluster j. The objective function that we are trying to optimize then is $$J = \sum_{i=1}^N \sum_{j=1}^K r_{ij}||\mathbf{x}_i - \mu_j||_2^2$$ Where r_{ij} is an indicator variable that takes on the value 1 if $x_i$ belongs to cluster j and 0 otherwise. 

#### Optimization (A First Look at the EM Algorithm)
The optimization of this objective function corresponds to a two step iterative process. The first step is to randomly initialize the K cluster centers $\mu_j$. Following this we want to determine $r_{ij}$. Since we are trying to minimize the above we simply set $r_{ij}=1$ for whichever cluster center j is closest. The following step is performed by taking the derivate with respect to each $\mu_k$ and setting equal to zero$$0 = 2 \sum_{i=1}^{N}r_{ik}(\mathbf{x_i}-\mu_k) \implies \mu_k = \frac{\sum_i r_{ik}\mathbf{x_i}}{\sum_i r_{ik}}$$ where $\mu_k$ is set to the mean of the points assigned to cluster k. The initial step here is called the Expectation Step and the second the Maximization Step.

### Gaussian Mixture Models
The basic idea behind the Gaussian Mixture Model is to simply model our data as having come from multiple Gaussians rather than just one. Then our distribution should take the form of $$P(X) = \sum_{i=1}^N \pi_i \mathcal{N}(\mu_i, \Sigma_i)$$ Where $\pi_i$ is the probability of having come from the ith Gaussian. To gain some more insight into what's going on we introduce a new latent variable $z \in \mathbb{R}^N$ where only one of the element $z_i = 1$ and the rest are 0. The marginal distribution over z is equal to the mixing coefficients $P(z_i = 1)=\pi_i$ which can be writting as $$p(\mathbf{z}) = \prod_{i=1}^N \pi_i^{z_i}$$ and the conditional distribution of x given a value of z $$P(\mathbf{x}|z_i=1)=\mathcal{N}(\mu_i,\Sigma_i)$$ which also can be written as $$P(\mathbf{x}|\mathbf{z})=\prod_{i=1}^{N}\mathcal{N}(\mu_i, \Sigma_i)^{z_i}$$ This just being equal to the ith Gaussian. From here we can obtain our marginal distribution $$P(x)=\sum_{\mathbf{z}}P(\mathbf{x}|\mathbf{z})P(\mathbf{z})=\sum_{i=1}^{N} \pi_i \mathcal{N}(\mu_i, \Sigma_i)$$ for the N possible states of z.
<br> Another quantity of interest is the posterior probability of z given x, called the responsibility of component k for explaining the observation x. $$\gamma(z_i) := P(z_i=1|\mathbf{x}) = \frac{P(\mathbf{x}|z_i = 1)P(z_i = 1)}{\sum_{j=1}^{N}P(\mathbf{x}|z_j = 1)P(z_j = 1)} = \frac{\pi_i\mathcal{N}(\mu_i, \Sigma_i)}{\sum_{j=1}^{N}\pi_j\mathcal{N}(\mu_j, \Sigma_j)}$$

#### Optimization (EM Algorithm for Mixtures of Gaussians)

First lets take a look at the form of the log-likelihood. $$log(P(X|\pi, \mu, \Sigma)) = \sum_{n=1}^N \log(\sum_{k=1}^K\pi_k\mathcal{N}(\mathbf{x_n}|\mu_k, \Sigma_k))$$ This is difficult to maximize due to the summation within the log. Let's take derivatives and see whats up. First with respect to $\mu_k$. $$0 = \sum_{n=1}^N \frac{\pi_k\mathcal{N}(\mu_k, \Sigma_k)}{\sum_j \pi_j\mathcal{N}(\mu_j, \Sigma_j)}\Sigma_k(\mathbf{x_n}-\mu_k) $$ Where the first term inside the summation is $\gamma(z_k)$ the responsibility of the kth Gaussian. $$\mu_k = \frac{1}{N_k}\sum_{n=1}^N \gamma(z_{nk})\mathbf{x}_n$$ Where, $$N_k = \sum_{n=1}^N \gamma(z_{nk})$$
So $\mu_k$ is a weighted mean of the data points, where each point is weighted by the posterior probability that component k was "responsible" for generating data point $\mathbf{x}_n$. We continue by doing the same thing for $\Sigma$ giving us, $$\Sigma_k = \frac{1}{N_k}\sum_{n=1}^N\gamma(z_{nk})(\mathbf{x}-\mu_k)(\mathbf{x}-\mu_k)^T$$
Same idea here but with the saple covariance matrix. Finally we want to maximize with respect to $\pi_k$ using the constraint that $\sum_{k=1}^{K}\pi_k =1$ sounds like a good time for lagrange multipliers. $$\log(P(X|\pi, \mu, \Sigma)) + \lambda(\sum_{k=1}^K \pi_k - 1)$$ This gives us, $$\pi_k = \frac{N_k}{N}$$ or the average responsibilities for the kth component. 

So the way we actually optimize this is through the following procedure, 
   1. Randomly initialize the parameters, $\pi, \mu, \Sigma$
   2. Evalutate the log likelihood given those parameters and the responsibilies given those parameters. 
   3. evaluate $\mu^{new}, \Sigma^{new}, \pi^{new}$ via the above equations.
   4. Set them to our new parameters and evaluate log likelihood for convergence. 
   5. repeat steps 2-5.

### Continuous Latent Variables, PCA, Autoencoders, Normalizing Flows and some more EM 


#### PCA(Three Perspectives)
PCA is a dimensionality reduction technique with 3 very nice interpretations that converge on the same solution.
1. The first of these is the least squares perspective which, while interesting in its connection with the common error metric I will not go into much depth on as I find the other approaches much more interesting. In general the least squares perspective on PCA is to find the projections of the data that minimize the projection error in terms of least squares between the projected points and the original points. 
2. The second approach is the maximum variance perspective. Given a number of dimensions that we want to reduce our data to we want to find the projection onto those dimensions that preserve the maximum amount of variance of our data. We start with just the first dimension. That is we want to find a projection vector $u_1 \in \mathbb{R}^p$ where $p < D$ such that the variance of the projected data $var(u_1X)=\sum_{i=1}^N(u_1^T\mathbf{x_i}-u_1^T\mathbf{\bar{x}})^2 = u_1^TSu_1$ is maximized with respect to u_1, where S is the sample covariance matrix. To make sure we the magnitude of u_1 doesn't go to infinity(we are concerned with the direction only), we enforce that the vectors be normalized, that $u_1^Tu_1 = 1$ using this constraint we can set up an optimization problem with lagrange multipliers(Who knew the worst topic for me in multivariable calc would be the one I used the most). $$\underset{u_1}{\mathrm{argmin}} \hspace{2mm} (u_1^TSu_1 - \lambda(1-u_1^Tu_1))$$
$$\implies 0 = 2Su_1 - 2 \lambda u_1 \implies Su_1 = \lambda u_1$$
And just like that we see that $u_1$ must be an eigenvector of S and by left multiplying by $u_1^T$ we get $$u_1^TSu_1 = \lambda$$ that is the variance is going to be equal to the eigenvalue. Therefore by finding the largest eigenvalue (And thus the highest variance) we can use the corresponding eigenvector as the projection of maximum variance. 
3. The third method Probabilistic PCA makes use of the construction of a generative probabilistic model and continuous latent variables. Here the continuous latent variable Z is a lower dimensional manifold that the data "actually" lives in. The data is corrupted by some gaussian noise and projected into higher dimensions. The generative model looks like, $$\mathbf{x} = \mathbf{W}\mathbf{z} + \mu + \epsilon, \epsilon \sim \mathcal{N}(0, \sigma^2I), \mathbf{z} \sim \mathcal{N}(0,I) \implies \mathbf{x}|\mathbf{z} \sim \mathcal{N}(\mathbf{Wz}+\mu, \sigma^2I) $$
Thankfully we have a bunch of Gaussians so computations be easy(or at least doable). 
$$P(\mathbf{x}) = \int P(\mathbf{x}|\mathbf{z})P(\mathbf{z})dz \implies \mathbf{x} \sim \mathcal{N}(\mu, \mathbf{W}^T\mathbf{W} + \sigma^2I)$$
Using this we can also evaluate the posterior of z using bayes rule as well as a natural likelihood parameter estimation framework. The results are what we would expect from the previous PCA methods. $$\mu_{ML} = \bar{\mathbf{x}},  \hspace{2mm} W_{ML} = \mathbf{U}_M(\mathbf{L}_M-\sigma^2I)^{1/2}\mathbf{R}, \hspace{2mm} \sigma^2_{ML} = \frac{1}{D-M} \sum_{i=M+1}^D\lambda_i$$


#### Autoencoders 
Autoencoders are a way to use neural networks for linear and non-linear dimensionality reduction. The basic idea is to train a multi-layer neural network split into two parts the encoder and the decoder. The job of the encoder is to transform the data into a lower dimensional representation while the encoder transforms it back into the original dimensionality. By minimizing the reconstruction error in this process we can the output of the encoder is a non-linear PCA.